In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import pdb
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
# import pickle
import torch
import torch.nn.functional as F
import torch.utils.data as data
# import pdb
from torch.autograd import Variable
# import argparse
##
# import matplotlib.pyplot as plt
import os
import numpy as np
import torch
# from tqdm import tqdm
# import argparse
# import json
import numpy as np
import os
import torch
# from datetime import datetime
# from pathlib import Path
##
# from sklearn import metrics
# import pdb
# from sklearn import metrics
# import argparse
# import json
import numpy as np
import os
import torch
from pathlib import Path
from sklearn import metrics
from torch.autograd import Variable

INPUT_DIM = 224
MAX_PIXEL_VAL = 255
MEAN = 58.09
STDDEV = 49.73


def preprocess(series):
        pad = int((series.shape[2] - INPUT_DIM)/2)
        series = series[:,pad:-pad,pad:-pad]
        series = (series-np.min(series))/(np.max(series)-np.min(series))*MAX_PIXEL_VAL
        series = (series - MEAN) / STDDEV
        series = np.stack((series,)*3, axis=1)
        series_float = torch.FloatTensor(series)
        return series_float

def get_study(axial_path, sagit_path, coron_path):
    vol_axial = np.load(axial_path)
    vol_sagit = np.load(sagit_path)
    vol_coron = np.load(coron_path)
    # axial
    vol_axial_tensor = preprocess(vol_axial)
    # sagittal
    vol_sagit_tensor =preprocess(vol_sagit)
    # coronal
    vol_coron_tensor = preprocess(vol_coron)

    return {"axial": vol_axial_tensor,
            "sagit": vol_sagit_tensor,
            "coron": vol_coron_tensor}


def get_prediction(model, tensors, abnormality_prior=None):
  vol_axial = tensors["axial"]
  vol_sagit = tensors["sagit"]
  vol_coron = tensors["coron"]

  if torch.cuda.is_available():
    vol_axial = tensors["axial"].cuda()
    vol_sagit = tensors["sagit"].cuda()
    vol_coron = tensors["coron"].cuda()

  vol_axial = Variable(vol_axial)
  vol_sagit = Variable(vol_sagit)
  vol_coron = Variable(vol_coron)

  logit = model.forward(vol_axial, vol_sagit, vol_coron)
  pred = torch.sigmoid(logit)
  pred_npy = pred.data.cpu().numpy()[0][0]

  if abnormality_prior:
        pred_npy = pred_npy * abnormality_prior
  return pred_npy

In [ ]:
class MRI_alex(nn.Module):
  def __init__(self, training=True):
    super().__init__()
    self.axial_net = models.alexnet(pretrained=training)
    self.sagit_net = models.alexnet(pretrained=training)
    self.coron_net = models.alexnet(pretrained=training)

    self.gap_axial = nn.AdaptiveAvgPool2d(1)
    self.gap_sagit = nn.AdaptiveAvgPool2d(1)
    self.gap_coron = nn.AdaptiveAvgPool2d(1)
    self.classifier = nn.Linear(3*256, 1)

  def forward(self,vol_axial, vol_sagit, vol_coron):
    vol_axial = torch.squeeze(vol_axial, dim=0)
    vol_sagit = torch.squeeze(vol_sagit, dim=0)
    vol_coron = torch.squeeze(vol_coron, dim=0)

    vol_axial = self.axial_net.features(vol_axial)
    vol_sagit = self.sagit_net.features(vol_sagit)
    vol_coron = self.coron_net.features(vol_coron)

    vol_axial = self.gap_axial(vol_axial).view(vol_axial.size(0), -1)
    x = torch.max(vol_axial, 0, keepdim=True)[0]

    vol_sagit = self.gap_sagit(vol_sagit).view(vol_sagit.size(0), -1)
    y = torch.max(vol_sagit, 0, keepdim=True)[0]

    vol_coron = self.gap_coron(vol_coron).view(vol_coron.size(0), -1)
    z = torch.max(vol_coron, 0, keepdim=True)[0]

    w = torch.cat((x, y, z), 1)
    out = self.classifier(w)
    return out

In [ ]:
print(torch.cuda.is_available())

In [ ]:
model_ab=MRI_alex(False)
model_path=""
if torch.cuda.is_available():
  model_ab.load_state_dict(torch.load(model_path,map_location=torch.device('cuda')))
else:
  model_ab.load_state_dict(torch.load(model_path,map_location=torch.device('cpu')))

In [ ]:
#get_prediction(model_ab,get_study("/content/drive/My Drive/MRNet-v1.0/valid/axial/1135.npy","/content/drive/My Drive/MRNet-v1.0/valid/sagittal/1135.npy","/content/drive/My Drive/MRNet-v1.0/valid/coronal/1135.npy"))
#get_prediction(model_ab,get_study("E:\\fci\\grad project\\I-MRI\\knee\\MRNet-v1.0\\valid\\axial\\1135.npy","E:\\fci\\grad project\\I-MRI\\knee\\MRNet-v1.0\\valid\\sagittal\\1135.npy","E:\\fci\\grad project\\I-MRI\\knee\\MRNet-v1.0\\valid\\coronal\\1135.npy"))